In [2]:
import requests
import re
from bs4 import BeautifulSoup
import json
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def clean_html(body):
    try:
        soup = BeautifulSoup(body, 'html.parser')
        a = soup.get_text(' ').replace('\n', '').lower()
        b = re.sub(r"[^a-z ]+","", a)
        c = ' '.join(b.split())
        return c
    except:
        return ''

In [4]:
df = pd.read_json("/dlabdata1/lugeon/websites_1000_5cat_html.json.gz", lines=True)

In [5]:
df.head()

,url,html,errcode,cat
0,http://www.360plex.com/news/go/127581/,,200.0,Arts
1,http://www.allwatchers.com/topics/info_4024.asp,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",404.0,Arts
2,http://www.opmden.com/,"<!DOCTYPE HTML PUBLIC ""-//IETF//DTD HTML 2.0//...",403.0,Arts
3,http://www.angelfire.com/ns/phil/,<!DOCTYPE HTML>\n\n<html>\n<head>\n<meta http-...,404.0,Arts
4,http://www.beatricegormley.com,<!DOCTYPE html>\n<html>\n<head>\n<meta content...,200.0,Arts


In [15]:
df = df[df.html.notnull()]
df = df[df.html != '']

In [16]:
df['html_clean'] = df.apply(lambda x: clean_html(x.html), axis=1)

In [17]:
df['html_len'] = df.apply(lambda x: len(x.html_clean.split()), axis=1)

In [18]:
df = df[df.html_len != 0]

In [19]:
html_counts = df.html_clean.value_counts().values[:200]
html_counts

array([98, 50, 42, 29, 25, 25, 22, 19, 17, 16, 14, 14, 12,  9,  9,  9,  9,
        8,  8,  7,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [20]:
len([x for x in html_counts if x != 1])

124

In [21]:
html_counts[123]

2

In [22]:
html_agg = df.html_clean.value_counts().keys().values

In [23]:
html_length = [len(body.split()) for body in html_agg]

In [24]:
errors = html_length[:124]

In [25]:
base = html_length[124:]

In [26]:
pd.Series(errors).describe()

count     124.000000
mean      194.975806
std       318.075884
min         1.000000
25%        19.000000
50%        66.500000
75%       265.000000
max      2317.000000
dtype: float64

In [27]:
pd.Series(base).describe()

count      3102.000000
mean        832.108640
std        4478.845666
min           1.000000
25%          59.000000
50%         257.000000
75%         653.000000
max      159757.000000
dtype: float64

In [28]:
html_error_list = html_agg[:124]

In [29]:
df['error'] = df.apply(lambda x: (x.html_clean in html_error_list), axis=1)

In [30]:
df

,url,html,cat,html_clean,html_len,error
1,http://www.allwatchers.com/topics/info_4024.asp,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",Arts,not found not found http error the requested r...,12,True
2,http://www.opmden.com/,"<!DOCTYPE HTML PUBLIC ""-//IETF//DTD HTML 2.0//...",Arts,forbidden forbidden you dont have permission t...,25,True
3,http://www.angelfire.com/ns/phil/,<!DOCTYPE HTML>\n\n<html>\n<head>\n<meta http-...,Arts,angelfire error angelfirecom professional but ...,22,True
4,http://www.beatricegormley.com,<!DOCTYPE html>\n<html>\n<head>\n<meta content...,Arts,beatrice gormley welcome to my website instagr...,814,False
5,http://vmnagaraj.tripod.com/,"<HTML>\n\n<HEAD>\n<TOBJ bgimage=""/imagelib/sty...",Arts,pannagasayanas hindustani music page welcome t...,340,False
...,...,...,...,...,...,...
4995,http://www.space-explorers.com/,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",Science,rocketplane global inc,3,False
4996,http://cse.nias.affrc.go.jp/shignak/chromosome...,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",Science,cse redirect information the domain has shifte...,118,False
4997,http://www.eea.europa.eu/,\n\n\n\n\n\n<!DOCTYPE html>\n\n<!-- paulirish....,Science,european environment agencys home page europea...,3904,False
4998,http://www.goes.noaa.gov/index.html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",Science,noaa goes geostationary satellite server pleas...,377,False


In [31]:
df.error.value_counts()

False    3102
True      721
Name: error, dtype: int64

In [32]:
df[df.error].html_len.describe()

count     721.000000
mean       95.636616
std       219.304142
min         1.000000
25%        13.000000
50%        24.000000
75%        58.000000
max      2317.000000
Name: html_len, dtype: float64

In [33]:
df[~df.error].html_len.describe()

count      3102.000000
mean        832.108640
std        4478.845666
min           1.000000
25%          59.000000
50%         257.000000
75%         653.000000
max      159757.000000
Name: html_len, dtype: float64

In [34]:
a = [x.split() for x in df[df.error].html_clean.values]
err_words = np.concatenate(a)

In [35]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/lugeon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
err_words_wo_stopwords = [w for w in err_words if w not in stopwords]

In [37]:
from collections import Counter 

In [38]:
Counter = Counter(err_words_wo_stopwords)

In [39]:
most_occur = Counter.most_common(10)

In [40]:
most_occur

[('found', 533),
 ('page', 415),
 ('error', 363),
 ('us', 283),
 ('web', 246),
 ('ft', 240),
 ('lbs', 240),
 ('site', 231),
 ('contact', 229),
 ('du', 228)]

In [44]:
url = "http://www.allwatchers.com/topics/info_4024.asp"
r = requests.get(url)

In [46]:
soup = BeautifulSoup(r.text, 'html.parser')

In [47]:
soup.get_text()

'\nNot Found\n\nNot Found\nHTTP Error 404. The requested resource is not found.\n\n'

In [52]:
r.status_code

404